In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana202'

# データ抽出
> ・特徴量結合データ（ana201_all_FTR_df）

In [2]:
all_df = pd.read_pickle(data_path / "ana201_all_FTR_df.pkl")

In [3]:
all_df.head(10)

,id,gameID,inning,batter,pitcher,y,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_cnt_B,FTR_cnt_O,FTR_cnt_S,FTR_pting_cnt,FTR_inning,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,...,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
0,0,20202173,1回表,ピレラ,今永 昇太,0.0,False,False,False,False,False,1,0,0,0,1,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.538462,0.384615,0.0,0.153846,0.0,0.000000,0.000000,0.0,0.0,0.0
1,1,20202173,1回表,ピレラ,今永 昇太,1.0,False,False,False,False,False,1,1,0,0,2,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.538462,0.384615,0.0,0.153846,0.0,0.000000,0.000000,0.0,0.0,0.0
2,2,20202173,1回表,ピレラ,今永 昇太,0.0,False,False,False,False,False,1,1,0,1,3,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.538462,0.384615,0.0,0.153846,0.0,0.111111,0.111111,0.0,0.0,0.0
3,3,20202173,1回表,ピレラ,今永 昇太,2.0,False,False,False,False,False,1,2,0,1,4,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.538462,0.384615,0.0,0.153846,0.0,0.000000,0.000000,0.0,0.0,0.0
4,4,20202173,1回表,ピレラ,今永 昇太,4.0,True,True,False,False,False,1,2,0,2,5,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.538462,0.384615,0.0,0.153846,0.0,0.500000,0.500000,0.0,0.0,0.0
5,5,20202173,1回表,菊池 涼介,今永 昇太,2.0,False,False,False,False,False,1,0,0,0,1,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.214286,0.166667,0.023810,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.166667,0.166667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
6,6,20202173,1回表,菊池 涼介,今永 昇太,3.0,False,False,False,False,False,1,0,0,1,2,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.214286,0.166667,0.023810,...,0.538462,0.461538,0.076923,0.153846,0.0,0.538462,0.461538,0.0,0.076923,0.0,0.8,0.6,0.2,0.4,0.0,0.166667,0.166667,0.0,0.000000,0.0,0.111111,0.111111,0.0,0.0,0.0
7,7,20202173,1回表,西川 龍馬,今永 昇太,1.0,False,False,False,False,False,1,0,1,0,1,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,...,0.538462,0.461538,0.076923,0.153846,0.0,0.200000,0.100000,0.1,0.100000,0.0,0.8,0.6,0.2,0.4,0.0,0.166667,0.166667,0.0,0.000000,0.0,0.076923,0.076923,0.0,0.0,0.0
8,8,20202173,1回表,西川 龍馬,今永 昇太,2.0,False,False,False,False,False,1,0,1,1,2,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,...,0.538462,0.461538,0.076923,0.153846,0.0,0.200000,0.100000,0.1,0.100000,0.0,0.8,0.6,0.2,0.4,0.0,0.166667,0.166667,0.0,0.000000,0.0,0.200000,0.000000,0.0,0.2,0.0
9,9,20202173,1回表,西川 龍馬,今永 昇太,0.0,False,False,False,False,False,1,0,1,2,3,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,...,0.538462,0.461538,0.076923,0.153846,0.0,0.200000,0.100000,0.1,0.100000,0.0,0.8,0.6,0.2,0.4,0.0,0.166667,0.166667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0


# データ加工
> 各イニングの結果（最後）のデータを作成 <br>
> ヒット予測モデルは、イニングデータを使用する <br>

In [4]:
# イニングの結果を残す
dup = all_df.duplicated(
    
                # 重複対象：イニング単位
                subset=['gameID', 'inning', 'batter'],

                # 削除するルール：最後の行（イニングの結果）を残す
                keep='last'
)

In [5]:
# 重複した最後の行（FALSE）だけ残す
all_dup_df = all_df[dup == 0]

In [6]:
# 確認用
all_dup_df.head(100)

,id,gameID,inning,batter,pitcher,y,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_cnt_B,FTR_cnt_O,FTR_cnt_S,FTR_pting_cnt,FTR_inning,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,...,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
4,4,20202173,1回表,ピレラ,今永 昇太,4.0,True,True,False,False,False,1,2,0,2,5,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.500000,0.500000,0.000000,0.0,0.000000
6,6,20202173,1回表,菊池 涼介,今永 昇太,3.0,False,False,False,False,False,1,0,0,1,2,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.214286,0.166667,0.023810,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.0,0.000000
12,12,20202173,1回表,西川 龍馬,今永 昇太,3.0,False,False,False,False,False,1,2,1,2,6,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.200000,0.100000,0.100000,0.100000,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
17,17,20202173,1回表,鈴木 誠也,今永 昇太,1.0,False,False,False,False,False,1,2,2,2,5,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.325000,0.150000,0.025000,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.0,0.000000
18,18,20202173,1回裏,梶谷 隆幸,大瀬良 大地,3.0,False,False,False,False,False,1,0,0,0,1,1,0.269542,0.188679,0.053908,0.000000,0.026954,0.288889,0.200000,0.044444,...,0.500000,0.388889,0.055556,0.000000,0.055556,0.411765,0.411765,0.000000,0.000000,0.000000,0.444444,0.222222,0.111111,0.0,0.111111,0.444444,0.333333,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
19,19,20202173,1回裏,乙坂 智,大瀬良 大地,3.0,False,False,False,False,False,1,0,1,0,1,1,0.269542,0.188679,0.053908,0.000000,0.026954,0.166667,0.166667,0.000000,...,0.500000,0.388889,0.055556,0.000000,0.055556,0.411765,0.411765,0.000000,0.000000,0.000000,0.444444,0.222222,0.111111,0.0,0.111111,0.444444,0.333333,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
24,24,20202173,1回裏,ソト,大瀬良 大地,3.0,False,False,False,False,False,1,3,2,1,5,1,0.269542,0.188679,0.053908,0.000000,0.026954,0.295455,0.204545,0.068182,...,0.500000,0.388889,0.055556,0.000000,0.055556,0.166667,0.055556,0.055556,0.000000,0.055556,0.444444,0.222222,0.111111,0.0,0.111111,0.444444,0.333333,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
26,26,20202173,2回表,メヒア,今永 昇太,4.0,True,True,False,False,False,1,0,0,1,2,2,0.269122,0.184136,0.036827,0.005666,0.042493,0.121212,0.090909,0.000000,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.111111,0.111111,0.000000,0.0,0.000000
33,33,20202173,2回表,會澤 翼,今永 昇太,1.0,False,False,False,False,False,1,3,0,2,7,2,0.269122,0.184136,0.036827,0.005666,0.042493,0.171429,0.142857,0.028571,...,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.00

# データ保存

In [7]:
all_dup_df.to_pickle(data_path / ("%s_all_FTR_dup_df.pkl" % prefix))